In [1]:
 !pip install mlxtend
 !pip install pandas 
 !pip install pprintpp

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   -------------------------------------- - 1.3/1.4 MB 7.5 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 5.8 MB/s eta 0:00:00
  Using cached pprintpp-0.4.0-py2.py3-none-any.whl.metadata (7.9 kB)
Using cached pprintpp-0.4.0-py2.py3-none-any.whl (16 kB)


In [2]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import association_rules, apriori

# Read Dataset

In [8]:
sales_reciepts = pd.read_csv ('dataset/201904 sales reciepts.csv')
sales_reciepts.head()

,transaction_id,transaction_date,transaction_time,sales_outlet_id,staff_id,customer_id,instore_yn,order,line_item_id,product_id,quantity,line_item_amount,unit_price,promo_item_yn
0,7,2019-04-01,12:04:43,3,12,558,N,1,1,52,1,2.50,2.50,N
1,11,2019-04-01,15:54:39,3,17,781,N,1,1,27,2,7.00,3.50,N
2,19,2019-04-01,14:34:59,3,17,788,Y,1,1,46,2,5.00,2.50,N
3,32,2019-04-01,16:06:04,3,12,683,N,1,1,23,2,5.00,2.50,N
4,33,2019-04-01,19:18:37,3,17,99,Y,1,1,34,1,2.45,2.45,N


In [9]:
product = pd.read_csv ('dataset/product.csv')
product.head()

,product_id,product_group,product_category,product_type,product,product_description,unit_of_measure,current_wholesale_price,current_retail_price,tax_exempt_yn,promo_yn,new_product_yn
0,1,Whole Bean/Teas,Coffee beans,Organic Beans,Brazilian - Organic,It's like Carnival in a cup. Clean and smooth.,12 oz,14.40,$18.00,Y,N,N
1,2,Whole Bean/Teas,Coffee beans,House blend Beans,Our Old Time Diner Blend,Out packed blend of beans that is reminiscent ...,12 oz,14.40,$18.00,Y,N,N
2,3,Whole Bean/Teas,Coffee beans,Espresso Beans,Espresso Roast,Our house blend for a good espresso shot.,1 lb,11.80,$14.75,Y,N,N
3,4,Whole Bean/Teas,Coffee beans,Espresso Beans,Primo Espresso Roast,Our primium single source of hand roasted beans.,1 lb,16.36,$20.45,Y,N,N
4,5,Whole Bean/Teas,Coffee beans,Gourmet Beans,Columbian Medium Roast,A smooth cup of coffee any time of day.,1 lb,12.00,$15.00,Y,N,N


# Data Wrangling

### Merge Data

In [10]:
dataset = pd.merge(sales_reciepts[['transaction_id','transaction_date', 'sales_outlet_id', 'customer_id', 'product_id', 'quantity']],
                      product[['product_id', 'product_category', 'product']],
                      on ='product_id', how = 'left' )
dataset.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product
0,7,2019-04-01,3,558,52,1,Tea,Traditional Blend Chai Rg
1,11,2019-04-01,3,781,27,2,Coffee,Brazilian Lg
2,19,2019-04-01,3,788,46,2,Tea,Serenity Green Tea Rg
3,32,2019-04-01,3,683,23,2,Coffee,Our Old Time Diner Blend Rg
4,33,2019-04-01,3,99,34,1,Coffee,Jamaican Coffee River Sm


### Remove Sizes

In [12]:
dataset[dataset['product'].str.contains('Dark chocolate')]['product'].unique()

array(['Dark chocolate Lg', 'Dark chocolate Rg', 'Dark chocolate'],
      dtype=object)

In [13]:
dataset['product'].nunique()

80

In [11]:
dataset['product'].str.contains('Dark chocolate')

0        False
1        False
2        False
3        False
4        False
         ...  
49889    False
49890    False
49891    False
49892    False
49893    False
Name: product, Length: 49894, dtype: bool

In [14]:
dataset['product']= dataset['product'].str.replace(' Rg','')
dataset['product']= dataset['product'].str.replace(' Sm','')
dataset['product']= dataset['product'].str.replace(' Lg','')

In [15]:
dataset['product'].nunique()

45

### Choose Product Subset

In [16]:
products_to_take = ['Cappuccino', 'Latte', 'Espresso shot',  \
                     'Dark chocolate','Sugar Free Vanilla syrup', 'Chocolate syrup',\
                    'Carmel syrup', 'Hazelnut syrup', 'Ginger Scone',  \
                    'Chocolate Croissant', 'Jumbo Savory Scone', 'Cranberry Scone', 'Hazelnut Biscotti',\
                    'Croissant', 'Almond Croissant', 'Oatmeal Scone', 'Chocolate Chip Biscotti',\
                    'Ginger Biscotti',\
                   ]

In [11]:
dataset = dataset[dataset['product'].isin(products_to_take)]

In [17]:
dataset['product'].isin(products_to_take)

0        False
1        False
2        False
3        False
4        False
         ...  
49889    False
49890    False
49891    False
49892    False
49893     True
Name: product, Length: 49894, dtype: bool

In [18]:
dataset['product'].nunique()

45

In [19]:
dataset[['product','product_category']].drop_duplicates().reset_index(drop=True)

,product,product_category
0,Traditional Blend Chai,Tea
1,Brazilian,Coffee
2,Serenity Green Tea,Tea
3,Our Old Time Diner Blend,Coffee
4,Jamaican Coffee River,Coffee
5,Ethiopia,Coffee
6,English Breakfast,Tea
7,Sustainably Grown Organic,Drinking Chocolate
8,Earl Grey,Tea
9,Cappuccino,Coffee


### Clean Transactions

In [20]:
dataset['transaction'] = dataset['transaction_id'].astype(str) +"_"+  dataset['customer_id'].astype(str)

In [21]:
num_of_items_for_each_transaction = dataset['transaction'].value_counts().reset_index()
valid_transactions = num_of_items_for_each_transaction[(num_of_items_for_each_transaction['count']>1)]['transaction'].tolist()
dataset = dataset[dataset['transaction'].isin(valid_transactions)]

In [22]:
dataset.shape

(33781, 9)

## Product Trends

In [23]:
dataset["product_category"].value_counts()

product_category
Coffee                11968
Tea                    8621
Bakery                 7406
Flavours               2247
Drinking Chocolate     2193
Coffee beans            565
Loose Tea               388
Branded                 245
Packaged Chocolate      148
Name: count, dtype: int64

In [24]:
dataset['product'].value_counts()

product
Ethiopia                        1722
Columbian Medium Roast          1690
Our Old Time Diner Blend        1654
Brazilian                       1610
Jamaican Coffee River           1591
Cappuccino                      1361
Latte                           1330
Sustainably Grown Organic       1177
Earl Grey                       1171
Peppermint                      1162
Morning Sunrise Chai            1160
Traditional Blend Chai          1134
Dark chocolate                  1114
Serenity Green Tea              1113
English Breakfast               1099
Lemon Grass                     1086
Spicy Eye Opener Chai           1084
Chocolate Croissant              945
Ginger Scone                     785
Jumbo Savory Scone               659
Espresso shot                    658
Cranberry Scone                  657
Hazelnut Biscotti                655
Croissant                        633
Scottish Cream Scone             631
Almond Croissant                 629
Chocolate Chip Biscotti       

In [28]:
dataset.groupby(['product','product_category']).count()

,,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,transaction
product,product_category,,,,,,,
Almond Croissant,Bakery,629,629,629,629,629,629,629
Brazilian,Coffee,1610,1610,1610,1610,1610,1610,1610
Brazilian - Organic,Coffee beans,68,68,68,68,68,68,68
Cappuccino,Coffee,1361,1361,1361,1361,1361,1361,1361
Carmel syrup,Flavours,561,561,561,561,561,561,561
Chili Mayan,Packaged Chocolate,50,50,50,50,50,50,50
Chocolate Chip Biscotti,Bakery,615,615,615,615,615,615,615
Chocolate Croissant,Bakery,945,945,945,945,945,945,945
Chocolate syrup,Flavours,569,569,569,569,569,569,569


# Popularity Recommendation Engine

In [26]:
popularity_recommendation = dataset.groupby(['product','product_category']).count().reset_index()

In [21]:
popularity_recommendation = popularity_recommendation[['product','product_category','transaction_id']]
popularity_recommendation = popularity_recommendation.rename(columns = {'transaction_id':'number_of_transactions'})

In [ ]:
popularity_recommendation.to_csv('api/recommendation_objects/popularity_recommendation.csv',index=False)

# Apriori Recommendation Engine

In [22]:
train_basket = (dataset.groupby(['transaction','product'])['product'].count().reset_index(name ='Count'))
train_basket.head(10)

,transaction,product,Count
0,1000_0,Dark chocolate,1
1,1000_0,Oatmeal Scone,1
2,1001_8306,Cappuccino,1
3,1001_8306,Carmel syrup,1
4,1002_0,Carmel syrup,1
5,1002_0,Dark chocolate,2
6,1002_0,Espresso shot,1
7,1002_0,Ginger Scone,1
8,1004_5383,Chocolate syrup,1
9,1004_5383,Latte,1


In [32]:
my_basket = train_basket.pivot_table(index='transaction', columns='product', values='Count', aggfunc='sum').fillna(0)
my_basket.head()

product,Almond Croissant,Cappuccino,Carmel syrup,Chocolate Chip Biscotti,Chocolate Croissant,Chocolate syrup,Cranberry Scone,Croissant,Dark chocolate,Espresso shot,Ginger Biscotti,Ginger Scone,Hazelnut Biscotti,Hazelnut syrup,Jumbo Savory Scone,Latte,Oatmeal Scone,Sugar Free Vanilla syrup
transaction,,,,,,,,,,,,,,,,,,
1000_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1001_8306,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1002_0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1004_5383,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1005_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0


In [33]:
def encode_units(x):
  if x <= 0:
    return 0
  if x >= 0:
    return 1

my_basket_sets = my_basket.applymap(encode_units)
my_basket_sets.head(10)

product,Almond Croissant,Cappuccino,Carmel syrup,Chocolate Chip Biscotti,Chocolate Croissant,Chocolate syrup,Cranberry Scone,Croissant,Dark chocolate,Espresso shot,Ginger Biscotti,Ginger Scone,Hazelnut Biscotti,Hazelnut syrup,Jumbo Savory Scone,Latte,Oatmeal Scone,Sugar Free Vanilla syrup
transaction,,,,,,,,,,,,,,,,,,
1000_0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
1001_8306,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1002_0,0,0,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0
1004_5383,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
1005_0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
1005_5559,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
1006_0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1007_8375,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
1008_0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0


In [34]:
frequent_items = apriori(my_basket_sets, min_support = 0.05,use_colnames = True)
frequent_items.head()

/home/abdullah/python_virtual_env/inkitt_env/lib/python3.8/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.115646,(Almond Croissant)
1,0.388889,(Cappuccino)
2,0.191232,(Carmel syrup)
3,0.112623,(Chocolate Chip Biscotti)
4,0.135676,(Chocolate Croissant)


In [35]:
rules_basket = association_rules(frequent_items, metric = "lift", min_threshold = 1)
rules_basket.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Cappuccino),(Almond Croissant),0.388889,0.115646,0.053288,0.137026,1.184874,0.008314,1.024775,0.255319
1,(Almond Croissant),(Cappuccino),0.115646,0.388889,0.053288,0.460784,1.184874,0.008314,1.133333,0.176432
2,(Dark chocolate),(Almond Croissant),0.277022,0.115646,0.057445,0.207367,1.793115,0.025409,1.115717,0.611791
3,(Almond Croissant),(Dark chocolate),0.115646,0.277022,0.057445,0.496732,1.793115,0.025409,1.436567,0.500152
4,(Latte),(Almond Croissant),0.382086,0.115646,0.054422,0.142433,1.231629,0.010235,1.031236,0.304358


In [ ]:
# Save Rules Basket
rules_basket.to_pickle('rules_basket.pkl')

In [36]:
# Get Example Recommendations
rules_basket[rules_basket['antecedents']=={'Latte'}].sort_values('confidence',ascending=False).head(50)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
70,(Latte),(Sugar Free Vanilla syrup),0.382086,0.200302,0.108844,0.284866,1.422182,0.032311,1.118249,0.480415
32,(Latte),(Carmel syrup),0.382086,0.191232,0.107710,0.281899,1.474121,0.034643,1.126260,0.520509
44,(Latte),(Chocolate syrup),0.382086,0.188964,0.103175,0.270030,1.428997,0.030974,1.111053,0.485842
68,(Latte),(Hazelnut syrup),0.382086,0.171580,0.101285,0.265084,1.544961,0.035727,1.127231,0.570848
52,(Latte),(Croissant),0.382086,0.114135,0.057067,0.149357,1.308605,0.013458,1.041407,0.381651
38,(Latte),(Chocolate Croissant),0.382086,0.135676,0.055178,0.144411,1.064381,0.003338,1.010209,0.097889
66,(Latte),(Ginger Scone),0.382086,0.133409,0.055178,0.144411,1.082472,0.004204,1.012860,0.123300
4,(Latte),(Almond Croissant),0.382086,0.115646,0.054422,0.142433,1.231629,0.010235,1.031236,0.304358
65,(Latte),(Ginger Biscotti),0.382086,0.106198,0.054044,0.141444,1.331890,0.013467,1.041053,0.403272
48,(Latte),(Cranberry Scone),0.382086,0.116024,0.051398,0.134520,1.159416,0.007067,1.021371,0.222518


### Save in Json Format

In [ ]:
product_categories = dataset[['product','product_category']].drop_duplicates().set_index('product').to_dict()['product_category']

In [ ]:
recommendations_json = {}

antecedents = rules_basket['antecedents'].unique()
for antecedent in antecedents:
    df_rec = rules_basket[rules_basket['antecedents']==antecedent]
    df_rec = df_rec.sort_values('confidence',ascending=False)
    key = "_".join(antecedent)
    recommendations_json[key] = []
    for _, row in df_rec.iterrows():
        rec_objects =row['consequents']
        for rec_object in rec_objects:
            already_exists = False
            for current_rec_object in recommendations_json[key]:
                if rec_object == current_rec_object['product']:
                    already_exists=True
            if already_exists:
                continue
            
            rec = {'product':rec_object, 
                   'product_category':product_categories[rec_object],
                   'confidence': row['confidence']
                  }
            recommendations_json[key].append(rec)

In [ ]:
import pprint
pprint.pp(recommendations_json)

In [ ]:
import json
with open('api/recommendation_objects/apriori_recommendations.json', 'w') as json_file:
    json.dump(recommendations_json, json_file)